In [1]:
import joblib
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
import shap


C:\Users\ali_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
data=pd.read_csv("data.csv")


In [29]:
data.drop(columns='round', inplace=True)
data.drop(columns='raceId', inplace=True)
data.drop(columns='points', inplace=True)
data.drop(columns='laps', inplace=True)
data.drop(columns='statusId', inplace=True)

In [37]:
data['pit_stop_time']=data['pit_stop_time']/1000 

In [38]:
data

,year,circuitId,driverId,constructorId,pit_stop_time,positionOrder,grid
0,2014,1,3,131,23.673,1,3
1,2014,1,3,131,24.331,1,3
2,2014,1,4,6,21.978,4,5
3,2014,1,4,6,22.887,4,5
4,2014,1,8,6,21.825,7,11
...,...,...,...,...,...,...,...
7974,2024,9,855,15,33.083,18,14
7975,2024,9,857,1,29.200,4,5
7976,2024,9,857,1,29.541,4,5
7977,2024,9,858,3,29.444,11,12


In [39]:
# Son 10 yıl verisini filtreleme
data = data[data["year"] >= 2014]

# Özellikler ve hedef değişken
X = data.drop(columns=["positionOrder", "year"]) 


#veriyi train test olarak bölelim

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

#modeli oluşturup eğitelim

rf_model= RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train,y_train)

#tahmin yapalım
y_pred = rf_model.predict(X_test)
print("classification report:\n", classification_report(y_test, y_pred))
print("accuracy:", accuracy_score(y_test, y_pred))

classification report:
               precision    recall  f1-score   support

           1       0.65      0.74      0.69        70
           2       0.58      0.60      0.59        88
           3       0.57      0.60      0.58        89
           4       0.59      0.68      0.63        77
           5       0.54      0.66      0.60        77
           6       0.65      0.52      0.58        85
           7       0.60      0.53      0.56        93
           8       0.65      0.67      0.66        79
           9       0.66      0.62      0.64        95
          10       0.55      0.56      0.56        75
          11       0.56      0.53      0.54        93
          12       0.62      0.66      0.64        97
          13       0.59      0.67      0.63        95
          14       0.60      0.65      0.63        89
          15       0.65      0.60      0.62        91
          16       0.52      0.59      0.55        76
          17       0.59      0.68      0.63        65
   

In [40]:
#modeli kaydedelim
joblib.dump(rf_model,'model.pkl')

['model.pkl']

In [41]:
explainer = shap.Explainer(rf_model)
shap_values_classification = explainer(X_test)
shap_values_classification

.values =
array([[[ 9.21479371e-04,  5.58326618e-04, -1.35303481e-03, ...,
         -9.01787152e-03, -1.42254483e-04,  7.20026679e-04],
        [-1.52935068e-02, -6.67033144e-03,  2.75231359e-03, ...,
          2.62152931e-03,  1.03765665e-03, -8.53821154e-04],
        [-1.28685185e-02, -2.61287415e-03, -1.29386515e-02, ...,
         -3.45054263e-03, -3.89617648e-03,  2.17870600e-03],
        [ 1.47131546e-03,  1.80596552e-03, -4.11223663e-03, ...,
         -1.49260840e-02, -8.53059169e-04,  9.49441014e-04],
        [-2.52020996e-02, -4.26345880e-02, -3.50189218e-02, ...,
          6.13439764e-03,  2.09778962e-04, -6.28434157e-03]],

       [[-8.47842525e-04, -1.49265011e-03, -3.34180562e-03, ...,
          6.15688525e-03,  3.88979827e-03, -7.56644637e-05],
        [-1.11462825e-02, -3.82764153e-03, -1.62581399e-04, ...,
         -2.21899657e-03, -5.22525928e-04,  3.00542877e-05],
        [-1.63642339e-02, -1.54329579e-02, -2.32632266e-02, ...,
          5.74664250e-03,  5.88144687e-03

In [42]:
dump(explainer, 'explainer.pkl')

['explainer.pkl']

In [36]:
shap.bar_plot(shap_values_classification)

TypeError: bad operand type for abs(): 'Explanation'